In [1]:
!pip install --upgrade keras
!pip install --upgrade tensorflow
!pip install --upgrade keras-svm

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.2.4)
Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (1.13.1)
Requirement already up-to-date: keras-svm in /usr/local/lib/python3.6/dist-packages (1.0.0b10)


In [2]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [0]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [0]:
from keras.utils import to_categorical

img_rows, img_cols = 28, 28
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

num_classes = 10

In [5]:
from keras import layers
from keras.models import Sequential
from keras.losses import categorical_crossentropy
from keras_svm import ModelSVMWrapper

def build_model():
  model = Sequential()
  model.add(layers.Conv2D(32, kernel_size=(3, 3),
                        strides=(2, 2), 
                        activation='relu', 
                        input_shape=input_shape))
  model.add(layers.Conv2D(32, (3, 3), strides=(2, 2),
                        activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Flatten(name="intermediate_output"))
  model.add(layers.Dense(20, activation='relu'))
  model.add(layers.Dense(num_classes, activation='softmax'))
  
  model.compile(loss=categorical_crossentropy,
              optimizer='rmsprop',
              metrics=['accuracy'])
  return model

wrapper = ModelSVMWrapper(build_model())

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
accuracy = {
    "with_svm": [],
    "with_softmax": []
}

wrapper.fit(x_train, y_train, epochs=5, batch_size=64)
accuracy["with_svm"].append(wrapper.evaluate(x_test, y_test))
accuracy["with_softmax"].append(
    wrapper.model.evaluate(x_test, to_categorical(y_test))[1])

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
60000/60000 [==============================] - 6s 94us/step - loss: 0.4034 - acc: 0.8794
Epoch 2/5
60000/60000 [==============================] - 3s 54us/step - loss: 0.1301 - acc: 0.9599
Epoch 3/5
60000/60000 [==============================] - 3s 54us/step - loss: 0.0983 - acc: 0.9693
Epoch 4/5
60000/60000 [==============================] - 3s 54us/step - loss: 0.0804 - acc: 0.9749
Epoch 5/5
10000/10000 [==============================] - 0s 45us/step


In [7]:
print(accuracy)

{'with_svm': [0.98], 'with_softmax': [0.974]}
